In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="4,5,6,7"
import sys
sys.path.insert(0, os.path.abspath(os.path.join(os.path.dirname(__file__), '..')))
import tasks
from rouge_score import rouge_scorer
import numpy as np
import torch
torch.set_grad_enabled(False)
import json

In [2]:
import pickle
def write_list(a_list, file_name):
    # store list in binary file so 'wb' mode
    with open(file_name, 'wb') as fp:
        pickle.dump(a_list, fp)
        print('Done writing list into a binary file')
def read_list(file_name):
    # for reading also binary mode is important
    with open(file_name, 'rb') as fp:
        n_list = pickle.load(fp)
        return n_list

def read_chatgpt_results(task):
    retrieved_scores = read_list(f'chatgpt_retrieved_scores_{task}.p')
    retrieved_true_scores = read_list(f'chatgpt_retrieved_true_scores_{task}.p')
    queries = read_list(f'chatgpt_queries_{task}.p')
    answers = read_list(f'chatgpt_answers_{task}.p')
    chatgpt_true_scores = read_list(f'chatgpt_true_scores_{task}.p')
    chatgpt_answers = read_list(f'chatgpt_answers_{task}.p')
    chatgpt_passages = read_list(f'chatgpt_passages_{task}.p')
    chatgpt_semantics = read_list(f'chatgpt_semantics_{task}.p')
    chatgpt_occurances = read_list(f'chatgpt_occurances_{task}.p')
    chatgpt_semantic_ids = read_list(f'chatgpt_semantic_ids_{task}.p')
    chatgpt_probs = read_list(f'chatgpt_probs_{task}.p')
    feasibilities = read_list(f'chatgpt_feasibilities_{task}.p')
    
    retrieved_scores_unc = read_list(f'chatgpt_retrieved_scores_unc_{task}.p')
    retrieved_true_scores_unc = read_list(f'chatgpt_retrieved_true_scores_unc_{task}.p')
    queries_unc = read_list(f'chatgpt_queries_unc_{task}.p')
    answers_unc = read_list(f'chatgpt_answers_unc_{task}.p')
    passages_unc = read_list(f'chatgpt_passages_unc_{task}.p')
    chatgpt_true_scores_unc = read_list(f'chatgpt_true_scores_unc_{task}.p')
    chatgpt_answers_unc = read_list(f'chatgpt_answers_unc_{task}.p')
    chatgpt_occurances_unc = read_list(f'chatgpt_occurances_unc_{task}.p')
    chatgpt_semantic_ids_unc = read_list(f'chatgpt_semantic_ids_unc_{task}.p')
    chatgpt_probs_unc = read_list(f'chatgpt_probs_unc_{task}.p')
    
    return feasibilities, retrieved_scores, retrieved_true_scores, queries, answers, chatgpt_true_scores, chatgpt_answers, chatgpt_passages, chatgpt_semantics, chatgpt_occurances, chatgpt_semantic_ids, chatgpt_probs, retrieved_scores_unc, retrieved_true_scores_unc, queries_unc, answers_unc, passages_unc, chatgpt_true_scores_unc, chatgpt_answers_unc, chatgpt_occurances_unc, chatgpt_semantic_ids_unc, chatgpt_probs_unc

In [67]:
task = 'squad1'

In [68]:
dataset_dpr = tasks.RQA_dpr(task=task)
elements = dataset_dpr.elements
all_queries = [element['question'] for element in elements]

In [69]:
feasibilities, retrieved_scores, retrieved_true_scores, queries, answers, chatgpt_true_scores, chatgpt_answers, chatgpt_passages, chatgpt_semantics, chatgpt_occurances, chatgpt_semantic_ids, chatgpt_probs, retrieved_scores_unc, retrieved_true_scores_unc, queries_unc, answers_unc, passages_unc, chatgpt_true_scores_unc, chatgpt_answers_unc, chatgpt_occurances_unc, chatgpt_semantic_ids_unc, chatgpt_probs_unc = \
        read_chatgpt_results(task)

In [70]:
query_certain = []
passage_certain = []
answers_certain = []
query_uncertain = []
passage_uncertain = []
answers_uncertain = []
final_index = all_queries.index(queries[-1])
for idx, query in enumerate(all_queries[:final_index+1]):
    if query in queries:
        query_certain.append(query)
        element = elements[idx]
        ctx = [psg['text'] for psg in element['positive_ctxs']]
        passage_certain.append(ctx[0])
    else:
        query_uncertain.append(query)
        element = elements[idx]
        ctx = [psg['text'] for psg in element['positive_ctxs']]
        if len(ctx) == 0:
            continue
        passage_uncertain.append(ctx[0])

In [71]:
certain = [{'sentence': query+'[SEP]'+passage, 'label':1} for query, passage in zip(query_certain, passage_certain)]

In [72]:
uncertain = [{'sentence': query+'[SEP]'+passage, 'label':0} for query, passage in zip(query_uncertain, passage_uncertain)]

In [73]:
all = certain + uncertain

In [74]:
with open(f"certain_uncertain_{task}.json", "a") as f:
    for line in all:
        json.dump(line, f)
        f.write('\n')